#### Paleo Analysis and Parsing of the Paleontological Database
Data extracted from *The Paleobiology Database*															


In [78]:
# library imports
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mpl
import requests as rq
import io

In [84]:
# Scraping data from the Paleobiology Database
url = rq.get('https://paleobiodb.org/data1.2/occs/taxa.csv?base_name=dinosauria&rank=max_genus&taxon_status=accepted&max_ma=252&min_ma=65&show=app,size,class').content
url2 = rq.get("http://paleobiodb.org/data1.2/occs/taxa.txt?datainfo&rowcount&base_name=pterosauria&rank=max_order&taxon_status=accepted&max_ma=252&min_ma=65&show=app,size,class").content
dinos = pd.read_csv(io.StringIO(url.decode('utf-8'))).drop(columns=['extant_size', 'orig_no', 'taxon_no', 'parent_no', 'record_type', 'flags', 'difference', 'accepted_no', 'accepted_rank', 'accepted_name', 'container_no', 'reference_no', 'is_extant', 'phylum', 'type_taxon', 'firstapp_min_ma', 'lastapp_max_ma'])

dinos.columns = ['Rank', 'Name', 'Number of Occurrences', 'First Appearance MYA (Max)', 'Last Appearance MYA (Min)', 'Early Interval', 'Late Interval', 'Taxon Size', 'Class', 'Order', 'Family', 'Genus']
dinos.head()

,Rank,Name,Number of Occurrences,First Appearance MYA (Max),Last Appearance MYA (Min),Early Interval,Late Interval,Taxon Size,Class,Order,Family,Genus
0,genus,Ajkaceratops,2,86.3,83.6,Santonian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Ajkaceratops
1,species,Ajkaceratops kozmai,1,86.3,83.6,Santonian,NaN,1.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Ajkaceratops
2,genus,Turanoceratops,12,93.9,89.8,Turonian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Turanoceratops
3,species,Turanoceratops tardabilis,6,93.9,89.8,Turonian,NaN,1.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Turanoceratops
4,genus,Zuniceratops,6,93.9,89.8,Turonian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Zuniceratops


In [80]:
genus = dinos[dinos["Rank"] == 'genus'].drop(columns=['Genus'])
genus.head()

,Rank,Name,Number of Occurrences,First Appearance MYA (Max),Last Appearance MYA (Min),Early Interval,Late Interval,Taxon Size,Class,Order,Family
0,genus,Ajkaceratops,2,86.3,83.6,Santonian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED
2,genus,Turanoceratops,12,93.9,89.8,Turonian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED
4,genus,Zuniceratops,6,93.9,89.8,Turonian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED
6,genus,Cerasinops,6,83.6,72.1,Campanian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,Leptoceratopsidae
8,genus,Ferrisaurus,2,72.1,66.0,Maastrichtian,NaN,2.0,Ornithischia,NO_ORDER_SPECIFIED,Leptoceratopsidae


In [81]:
species = dinos[dinos["Rank"] == 'species'].drop(columns=['Taxon Size'])
species.head()

,Rank,Name,Number of Occurrences,First Appearance MYA (Max),Last Appearance MYA (Min),Early Interval,Late Interval,Class,Order,Family,Genus
1,species,Ajkaceratops kozmai,1,86.3,83.6,Santonian,NaN,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Ajkaceratops
3,species,Turanoceratops tardabilis,6,93.9,89.8,Turonian,NaN,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Turanoceratops
5,species,Zuniceratops christopheri,3,93.9,89.8,Turonian,NaN,Ornithischia,NO_ORDER_SPECIFIED,NO_FAMILY_SPECIFIED,Zuniceratops
7,species,Cerasinops hodgskissi,3,83.6,72.1,Campanian,NaN,Ornithischia,NO_ORDER_SPECIFIED,Leptoceratopsidae,Cerasinops
9,species,Ferrisaurus sustutensis,1,72.1,66.0,Maastrichtian,NaN,Ornithischia,NO_ORDER_SPECIFIED,Leptoceratopsidae,Ferrisaurus
